# Measure Planetary Radius from Your Own Image

This notebook guides you through measuring a planet's radius from your own horizon photograph. No configuration files needed!

**What you'll need:**
- A photo showing a planetary horizon (Earth, Mars, etc.)
- The altitude when the photo was taken (from GPS, flight data, or mission specs)
- About 5-10 minutes

**What you'll learn:**
- How to extract camera parameters automatically from EXIF data
- How to validate your measurement setup
- Three different methods for horizon detection:
  - **Manual annotation** (interactive, most reliable)
  - **ML segmentation** (automatic with Segment Anything Model)
  - **Gradient-field** (automatic, physics-based)
- How to interpret optimization results and uncertainties

Let's get started!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import planet_ruler.observation as pro
from planet_ruler.camera import create_config_from_image, extract_camera_parameters
from planet_ruler.validation import validate_limb_config
from planet_ruler.plot import (
    plot_3d_solution,
    plot_gradient_field_at_limb,
    plot_residuals,
    plot_gradient_field_quiver,
)
from planet_ruler.dashboard import OutputCapture
from planet_ruler.uncertainty import calculate_parameter_uncertainty
from planet_ruler.fit import format_parameter_result

# these are here for developers
%load_ext autoreload
%autoreload 2

## Step 1: Specify Your Image and Context

Point to your image file and provide the key information we need:
- **altitude_km**: How high was the camera when the photo was taken? (in kilometers)
- **planet**: Which planet are you measuring? ("earth", "mars", "jupiter", etc.)

The altitude is **critical** for accurate measurements. If you're not sure:
- For airplane photos: typical cruising altitude is 10-12 km
- For ISS photos: approximately 400-420 km  
- For high-altitude balloons: 30-40 km
- Check GPS data in your photo's EXIF metadata (we'll show this below)

In [ ]:
# ====== EDIT THESE VALUES ======
image_path = "path/to/your/horizon_photo.jpg"  # Your image file
image_path = "../../demo/images/2013-08-05_22-42-14_Wikimania.jpg"  # Challenging example -- use 10k altitude
image_path = "../../demo/images/50644513538_56228a2027_o.jpg"  # Easy example -- use 418kk altitude
altitude_m = 10_000  # Altitude in meters when photo was taken
altitude_m = 418_000
planet = "earth"  # Which planet: "earth", "mars", "jupiter", "saturn", etc.
# ===============================

## Step 2: Extract Camera Parameters

Planet Ruler can automatically extract camera information from your image's EXIF data. This includes:
- Focal length
- Sensor dimensions (from camera database or calculation)
- Image dimensions
- GPS altitude (if available)

Let's see what we can learn from your image:

In [ ]:
# Extract camera parameters from EXIF
camera_info = extract_camera_parameters(image_path)

print("=" * 60)
print("CAMERA PARAMETER EXTRACTION")

print("=" * 60)
print(f"\nDetected Camera: {camera_info.get('camera_model', 'Unknown')}")
print(f"Camera Type: {camera_info.get('camera_type', 'unknown')}")
print(f"Confidence Level: {camera_info.get('confidence', 'unknown')}")
print(
    f"\nImage Dimensions: {camera_info['image_width_px']} x {camera_info['image_height_px']} pixels"
)

if camera_info["focal_length_mm"]:
    print(f"\nFocal Length: {camera_info['focal_length_mm']:.2f} mm")
else:
    print("\nFocal Length: Not found in EXIF")

if camera_info["sensor_width_mm"]:
    print(f"Sensor Width: {camera_info['sensor_width_mm']:.2f} mm")
    if camera_info.get("sensor_width_min") and camera_info.get("sensor_width_max"):
        print(
            f"  Range: [{camera_info['sensor_width_min']:.2f}, {camera_info['sensor_width_max']:.2f}] mm"
        )
else:
    print("Sensor Width: Using defaults")

# Check for GPS altitude
from planet_ruler.camera import get_gps_altitude

gps_altitude = get_gps_altitude(image_path)
if gps_altitude:
    print(f"\n✓ GPS Altitude Found: {gps_altitude:.1f} meters")
    print(
        f"   Consider using this instead of your specified altitude: {altitude_m} meters"
    )
else:
    print(
        f"\nGPS Altitude: Not found - using your specified altitude of {altitude_m} meters"
    )

print("\n" + "=" * 60)

## Step 3: Generate Configuration

Now we'll automatically generate a complete configuration for your measurement using the extracted camera parameters and your inputs:

In [ ]:
# Create configuration automatically from image and your inputs
config = create_config_from_image(
    image_path=image_path,
    altitude_m=altitude_m,
    planet=planet,
    perturbation_factor=0.5,  # Perturb initial radius guess to avoid local minima
)

print("=" * 60)
print("AUTO-GENERATED CONFIGURATION")
print("=" * 60)
print(f"\nTarget Planet: {planet.title()}")
print(f"Altitude: {altitude_m} meters")
print(f"\nCamera Configuration:")
print(f"  Focal Length: {config['init_parameter_values']['f']*1000:.2f} mm")
if "w" in config["init_parameter_values"]:
    print(f"  Sensor Width: {config['init_parameter_values']['w']*1000:.2f} mm")
else:
    print(f"  Field of View: {config['init_parameter_values']['fov']:.1f}°")
print(f"\nInitial Parameter Guess:")
print(
    f"  Radius: {config['init_parameter_values']['r']/1000:.0f} km (perturbed from truth)"
)
print(f"  Altitude: {config['init_parameter_values']['h']/1000:.1f} km")
print(f"\n✓ Configuration ready for optimization")
print("=" * 60)

## Step 4: Validate Configuration

Before we start the measurement, let's validate that our configuration is internally consistent and well-formed. This checks:
- Initial parameter values are within their specified limits
- Orientation angle (theta) ranges are wide enough to avoid coupling issues
- Radius limits span a reasonable range for robust optimization

In [ ]:
print("=" * 60)
print("CONFIGURATION VALIDATION")
print("=" * 60)
print()

# Validate (strict=False will show warnings but not fail)
try:
    validate_limb_config(config, strict=True)
    print("✓ Configuration validation passed!")
    print("\nAll parameter bounds are reasonable and internally consistent.")
except AssertionError as e:
    print(f"Configuration issue detected:")
    print(f"   {e}")
    print("\nYou may want to adjust parameters before continuing.")

print("\n" + "=" * 60)

## Step 5: Load Image and View It

Now let's load your image into a `LimbObservation` object and take a look at what we're working with:

In [ ]:
# Create observation using auto-generated config
Obs = pro.LimbObservation(
    image_filepath=image_path, fit_config=config  # Using dict instead of file path
)

print(f"Image loaded: {Obs.image.shape[1]} x {Obs.image.shape[0]} pixels")
print(f"Free parameters: {Obs.free_parameters}")
print()

# Display the image
Obs.plot()

## Step 6: Choose Your Detection Method

Planet Ruler offers three approaches to horizon detection. Each has different trade-offs:

### Method 1: Manual Annotation (Recommended for beginners)
- **Interactive GUI**: Click points along the horizon
- **High accuracy**: You know better than any algorithm where the true horizon is
- **Works everywhere**: Effective even with clouds, atmospheric haze, or unusual features
- **Quick**: Usually takes less than a minute to click 10-20 points
- **Lightweight**: Minimal compute requirements

### Method 2: Gradient-Field Optimization (Automatic, lightweight)
- **Fully automatic**: No human input or detection needed
- **Physics-based**: Uses brightness gradients perpendicular to the limb
- **Multi-resolution**: Optimizes on progressively higher resolution images
- **Lightweight**: No ML models, modest memory footprint
- **Best for batch processing**: Efficient for processing many images
- **Best for clean horizons**: Works well with sharp Earth/planet boundaries
- **May struggle with**: Atmospheric layers, clouds, or complex lighting

### Method 3: ML Segmentation (Automatic but heavyweight)
- **Fully automatic**: Uses Segment Anything Model (SAM) to detect the horizon
- **Good for clear boundaries**: Works well when planet/space boundary is distinct
- **Heavy compute**: Requires ~2GB model download and significant GPU/CPU resources
- **May struggle with**: Subtle horizons, atmospheric gradients, or ambiguous features
- **Resource intensive**: Not ideal for batch processing due to model size

**Recommendation:**
- **First time?** → Try Method 1 (Manual)
- **Processing many images?** → Try Method 2 (Gradient-Field)
- **Have GPU and want automatic detection?** → Try Method 3 (ML Segmentation)

Choose one approach below and run the corresponding cell:

### Option 1: Manual Annotation

**Instructions:**
1. A window will open showing your image
2. Click along the horizon to place points (10-20 points is usually good)
3. Try to space them somewhat evenly across the visible horizon
4. When done, close the window
5. The notebook will automatically fit a smooth curve through your points

In [ ]:
# Run this cell for MANUAL annotation
detection_method = "manual"

print("Opening manual annotation tool...")
print("Click points along the horizon, then close the window when done.\n")

Obs.limb_detection = detection_method
Obs.detect_limb()

print("\n✓ Horizon annotation complete!")
print(
    f"   Detected {np.count_nonzero(~np.isnan(Obs.features['limb']))} points along the horizon"
)

# Show the detected limb
Obs.plot()

### Option 2: Gradient-Field Optimization

This method skips explicit horizon detection and instead optimizes parameters directly on the image using brightness gradients. It automatically uses multi-resolution optimization to avoid local minima.

**Advantages:**
- No model downloads needed
- Lightweight and efficient
- Ideal for batch processing many images
- Works well with clean planetary horizons

**Note:** If your image has complex atmospheric features or clouds near the horizon, manual annotation may work better.

In [ ]:
# Run this cell for GRADIENT-FIELD method (skip detection)
detection_method = "gradient-field"

Obs.limb_detection = detection_method
print("Using gradient-field method - no explicit limb detection needed.")
print("The optimizer will work directly with image gradients.")
print()

plot_gradient_field_quiver(Obs.image, step=2, image_smoothing=2.0, kernel_smoothing=8.0)

### Option 3: ML Segmentation (Segment Anything Model)

This uses Meta's Segment Anything Model (SAM) to automatically detect the horizon. 

**First time setup:**
- The model (~2GB) will be downloaded automatically on first use
- This may take a few minutes depending on your connection
- Subsequent uses will be faster as the model is cached

**Performance notes:**
- With GPU: Usually completes in 10-30 seconds
- CPU only: May take 1-5 minutes per image
- Not ideal for batch processing due to memory requirements

**When it works well:**
- Clear planet/space boundary
- Distinct color or brightness difference at horizon
- Uniform lighting

**When it may struggle:**
- Atmospheric haze that blurs the boundary
- Similar colors above and below horizon
- Complex cloud structures

**Note:** For batch processing, gradient-field (Option 2) is more efficient due to lower resource requirements.

In [ ]:
# Run this cell for ML SEGMENTATION
detection_method = "segmentation"

print("Starting ML segmentation...")
print("(First time: downloading ~2GB model - please be patient)\n")

Obs.limb_detection = detection_method
Obs.detect_limb(segmentation_method="sam", interactive=True, downsample_factor=5)

# Smooth the detected limb to remove jitter
Obs.smooth_limb(method="rolling-median", window_length=15)

print("\n✓ ML segmentation complete!")
print(
    f"   Detected {np.count_nonzero(~np.isnan(Obs.features['limb']))} points along the horizon"
)
print(
    "   Review the detection below - if it looks wrong, try manual annotation instead."
)

# Show the detected limb
Obs.plot()

## Step 7: Fit Planetary Radius

Now comes the main event: optimizing all free parameters to find the best-fit planetary radius.

**What's happening:**
- The optimizer searches through parameter space (radius, altitude, camera properties, orientation)
- It tries to match the predicted limb position to the observed horizon
- The dashboard shows real-time progress with adaptive refresh rates
- Multi-resolution stages help avoid getting stuck in local minima

**This may take a few minutes.** Watch the dashboard to see:
- Current parameter estimates and how they compare to the true values
- Loss function reduction (you want this decreasing steadily)
- Progress bars for each optimization stage
- Helpful warnings and hints

The cell below is configured differently depending on which method you chose:

In [ ]:
# Create output capture for displaying print statements in dashboard
capture = OutputCapture(max_lines=20, line_width=70)

if detection_method in ["manual", "segmentation"]:
    # Manual or ML segmentation: standard L1 loss on detected limb points
    method_name = (
        "manual annotation" if detection_method == "manual" else "ML segmentation"
    )
    print(f"Starting optimization with {method_name}...")
    print()

    with capture:
        Obs.fit_limb(
            minimizer="differential-evolution",
            max_iter=3000,
            verbose=True,
            dashboard=True,
            n_jobs=6,
            target_planet=planet,
            dashboard_kwargs={
                "output_capture": capture,
                "width": 80,
                "max_warnings": 5,
                "max_hints": 4,
                "min_message_display_time": 5.0,
            },
        )

elif detection_method == "gradient-field":
    # Gradient-field method: direct optimization on image gradients
    print("Starting gradient-field optimization...")
    print("Using multi-resolution strategy: coarse --> fine")
    print()

    with capture:
        Obs.fit_limb(
            minimizer="differential-evolution",
            loss_function="gradient_field",
            resolution_stages=[8, 4],
            image_smoothing=2.0,  # Remove high-frequency image artifacts
            kernel_smoothing=8.0,  # Smooth gradient field for stability
            minimizer_preset="scipy-default",
            prefer_direction=None,
            max_iter=300,  # 3000,
            verbose=True,
            dashboard=True,
            target_planet=planet,
            dashboard_kwargs={
                "output_capture": capture,
                "width": 80,
                "max_warnings": 5,
                "max_hints": 4,
                "min_message_display_time": 5.0,
            },
        )

print("\n✓ Optimization complete!")

## Step 8: Visual Check of Results

Let's take a look at the fitted solution. The predicted limb should closely match the actual horizon in your image.

In [ ]:
# Plot image with fitted limb
Obs.plot()

print(f"\nBest-fit radius: {Obs.radius_km:.1f} km")
print(f"Best-fit altitude: {Obs.altitude_km:.1f} km")

If we fit to the limb (using any method except gradient-field), we can take a look at the fit residuals.

In [ ]:
if detection_method != "gradient-field":
    plot_residuals(
        Obs,
        show_image=True,
        show_sparse_markers=True,
        image_alpha=0.6,
        figsize=(13, 8),
        band_size=30,
    )
else:
    print("No target -- no residuals!")

### For Gradient-Field Method: Check Gradient Directions

If you used the gradient-field method, we can visualize the brightness gradient vectors at the detected limb. 
A good fit shows:
- Strong gradients (long arrows)
- Perpendicular to the limb
- All pointing the same direction (inward or outward)

Run this cell only if you used gradient-field optimization:

In [ ]:
# Only run if using gradient-field method
if detection_method == "gradient-field":
    plot_gradient_field_quiver(
        Obs.image, step=2, image_smoothing=2.0, kernel_smoothing=8.0
    )

    fig, ax = plot_gradient_field_at_limb(
        y_pixels=Obs.features["fitted_limb"],
        image=Obs.image,
        image_smoothing=2.0,
        directional_smoothing=50,
        directional_decay_rate=0.15,
        sample_spacing=100,
        kernel_smoothing=8.0,
    )
    plt.show()
    print("\nArrows show gradient direction and strength at sampled points.")
    print("They should be perpendicular to the limb and point the same direction.")
else:
    print("Gradient field visualization only available for gradient-field method.")

## Step 9: 3D Visualization

Let's visualize the geometry in 3D to see how the camera, planet, and horizon relate:

In [ ]:
plot_3d_solution(**Obs.best_parameters)

## Step 10: Calculate Uncertainties

Now let's quantify our measurement uncertainty. Planet Ruler can estimate parameter uncertainties using several methods:

- **Population spread** (for differential-evolution): Uses final population distribution
- **Hessian approximation**: Fast analytical estimate from optimization curvature
- **Auto selection**: Automatically chooses the best method for your minimizer

The uncertainty reflects both measurement precision and systematic uncertainties in camera parameters.

In [ ]:
print("=" * 60)
print("MEASUREMENT RESULTS WITH UNCERTAINTIES")
print("=" * 60)
print()

# Radius uncertainty
radius_result = calculate_parameter_uncertainty(
    Obs, parameter="r", scale_factor=1000, method="auto"  # Convert m to km
)
print("RADIUS:")
print(format_parameter_result(radius_result, units="km"))
print(f"Method: {radius_result['method']}")
print()

# Altitude uncertainty
altitude_result = calculate_parameter_uncertainty(
    Obs, parameter="h", scale_factor=1000, method="auto"  # Convert m to km
)
print("ALTITUDE:")
print(format_parameter_result(altitude_result, units="km"))
print(f"Method: {altitude_result['method']}")
print()

print("=" * 60)

## Step 11: Compare to Known Values

If you're measuring Earth or another planet with a well-known radius, let's see how close we got:

In [ ]:
# Known planetary radii (km)
TRUE_RADII = {
    "earth": 6371,
    "mars": 3390,
    "jupiter": 69911,
    "saturn": 58232,
    "uranus": 25362,
    "neptune": 24622,
    "venus": 6052,
    "mercury": 2440,
    "moon": 1737,
    "pluto": 1188,
}

if planet.lower() in TRUE_RADII:
    true_radius = TRUE_RADII[planet.lower()]
    measured_radius = radius_result["value"]
    uncertainty = radius_result["uncertainty"]

    error_km = measured_radius - true_radius
    error_pct = (error_km / true_radius) * 100

    print("=" * 60)
    print(f"COMPARISON TO KNOWN {planet.upper()} RADIUS")
    print("=" * 60)
    print()
    print(f"Your measurement:  {measured_radius:.1f} ± {uncertainty:.1f} km")
    print(f"Known value:       {true_radius} km")
    print()
    print(f"Difference:        {error_km:+.1f} km ({error_pct:+.2f}%)")

    # Check if within uncertainty
    sigma_away = abs(error_km) / uncertainty if uncertainty > 0 else float("inf")
    print(f"Statistical:       {sigma_away:.1f}σ from true value")
    print()

    if abs(error_pct) < 10:
        print("✓ Excellent! Within 10% of the true value.")
    elif abs(error_pct) < 20:
        print("✓ Good! Within 20% of the true value.")
    else:
        print(f"Larger than expected error. Consider:")
        print("   - Is the altitude accurate?")
        print("   - Is the horizon clearly visible?")
        print("   - Did the optimization converge? (check dashboard)")

    print("\n" + "=" * 60)
else:
    print(f"No known radius for '{planet}' in database for comparison.")
    print(
        f"Your measured radius: {radius_result['value']:.1f} Â± {radius_result['uncertainty']:.1f} km"
    )

## Step 12: Summary and Next Steps

Congratulations! You've successfully measured a planetary radius from your own image.

**What you accomplished:**
- ✓ Extracted camera parameters from image metadata
- ✓ Validated the measurement configuration
- ✓ Detected the planetary horizon (manual, ML, or gradient-field)
- ✓ Optimized geometric parameters to find the best-fit radius
- ✓ Quantified measurement uncertainty

**Ways to improve your measurement:**
1. **Better altitude data**: GPS altitude from EXIF is most accurate
2. **Try all three methods**: Compare manual, ML segmentation, and gradient-field results
3. **Multiple images**: Average results from several photos
4. **Longer optimization**: Increase `max_iter` for more thorough parameter search
5. **Known camera**: If sensor dimensions are in the database, confidence is higher
6. **Validate ML results**: If using segmentation, always inspect with `Obs.plot()` before fitting

**Share your results!**
Consider sharing your findings with the planet-ruler community or using them in educational projects.

## Optional: Save Your Results

You can save the detected limb and best-fit parameters for later analysis:

In [ ]:
# Uncomment and customize to save results
# import json
# import numpy as np

# # Save detected limb
# if detection_method == "manual":
#     np.save("my_detected_limb.npy", Obs.features["detected_limb"])

# # Save fitted limb
# np.save("my_fitted_limb.npy", Obs.features["fitted_limb"])

# # Save best parameters
# with open("my_results.json", "w") as f:
#     json.dump({
#         "radius_km": Obs.radius_km,
#         "radius_uncertainty_km": radius_result['uncertainty'],
#         "altitude_km": Obs.altitude_km,
#         "method": detection_method,
#         "planet": planet,
#         "best_parameters": Obs.best_parameters
#     }, f, indent=2)

# print("✓ Results saved!")